In [1]:
import json
import glob
import numpy as np

In [2]:
embodied_carbon = {
    'p100': 16608.56,
    'v100': 24683.58,
    'a100': 41937.91
}

benchmark_paths = {
    'candle': '/work/li.baol/GIT/candle_benchmarks/benchmark_logs',
    'cv': '/work/li.baol/GIT/examples/imagenet/benchmark_logs',
    'nlp': '/work/li.baol/GIT/transformers/examples/pytorch/question-answering/benchmark_logs'
}

upgrade_options = [('p100', 'v100'), ('p100', 'a100'), ('v100', 'a100')]
with open('embodied_carbon_gCO2.json', 'w') as f:
    json.dump(embodied_carbon, f, indent=4)

In [3]:
perf_gain = {}

for option in upgrade_options:
    perf_gain[str(option)] = {} # {'p100-v100': {'cv': xxx}, ...}
    for k, v in benchmark_paths.items():
        base, upgrade = option[0], option[1] # p100, v100
        base_path = f'{v}/1x{base}'
        upgrade_path = f'{v}/1x{upgrade}'
        read_list = glob.glob(f'{base_path}/time_*.json')
        perf = {}
        for generation in [base, upgrade]:
            perf[generation] = []
            for read in read_list:
                if generation == upgrade:
                    filename = read.replace(base, upgrade)
                else:
                    filename = read
                with open(filename) as f:
                    load = json.load(f)
                perf[generation].append(np.mean(load[1:]))
        perf_diff = (np.array(perf[base]) - np.array(perf[upgrade])) / np.array(perf[base]) * 100
        perf_gain[str(option)][k] = np.mean(perf_diff)
with open('perf_gain.json', 'w') as f:
    json.dump(perf_gain, f, indent=4)

In [4]:
cpu_ratio = {} # {'cv': {'workload1': x}}
for k, v in benchmark_paths.items():
    cpu_ratio[k] = {}
    gpu_path = f'{v}/1xv100'
    cpu_path = f'{v}/1xcpu'
    read_list = glob.glob(f'{gpu_path}/carbon_*.json')
    for read in read_list:        
        with open(read) as f:
            load = json.load(f)
        gpu_energy = load['energy (kWh)']
        read = read.replace('v100', 'cpu')
        with open(read) as f:
            load = json.load(f)
        cpu_energy = load['energy (kWh)']
        workload = read.split('/')[-1]
        cpu_ratio[k][workload] = cpu_energy / gpu_energy
cpu_ratio

{'candle': {'carbon_combo.json': 0.4509085734727526,
  'carbon_nt3.json': 0.24721815143162584,
  'carbon_p1b2.json': 0.39437661150643727,
  'carbon_st1.json': 0.23193688027446757,
  'carbon_tc1.json': 0.24670639649130424},
 'cv': {'carbon_resnet50.json': 0.2792243535201363,
  'carbon_resnext50_32x4d.json': 0.27445470444060993,
  'carbon_shufflenet_v2_x1_5.json': 0.4659915655246899,
  'carbon_vgg19_bn.json': 0.2982994422791288,
  'carbon_vit_b_16.json': 0.24886130579300078},
 'nlp': {'carbon_bert-base-uncased.json': 0.23648722632610109,
  'carbon_distilbert-base-cased.json': 0.23989652036196052,
  'carbon_microsoft-mpnet-base.json': 0.2556783520503392,
  'carbon_roberta-base.json': 0.23512827429819785,
  'carbon_valhalla-bart-large-finetuned-squadv1.json': 0.2386524836683373}}

In [5]:
energy_per_run = {} # in kWh unit, per run, per GPU

daily_usage_ratio = .4
for option in upgrade_options:
    energy_per_run[option] = {} # {'p100-v100': {'cv': {'p100': [xx, xx], 'v100': [x], 'runs_per_day': [xx]}}, ...}
    for k, v in benchmark_paths.items(): # {candle: xxx, cv: xxx}
        base, upgrade = option[0], option[1] # p100, v100
        base_path = f'{v}/1x{base}'
        upgrade_path = f'{v}/1x{upgrade}'
        read_list = glob.glob(f'{base_path}/carbon_*.json')
        energy_per_run[option][k] = {}
        energy_per_run[option][k]['runs_per_day'] = []
        for generation in [base, upgrade]:
            energy_per_run[option][k][generation] = []            
            for read in read_list:
                if generation == upgrade:
                    filename = read.replace(base, upgrade)
                else:
                    filename = read
                with open(filename) as f:
                    load = json.load(f)
                total_energy = load['energy (kWh)']
                if generation == 'a100':
                    testcase = read.split('/')[-1]
                    ratio = cpu_ratio[k][testcase]
                    total_energy = total_energy / (1 - ratio)
                energy_per_run[option][k][generation].append(total_energy)
                if generation == base:
                    duration = load['duration (s)']
                    num_runs = 24 * 60 * 60 * daily_usage_ratio / duration
                    energy_per_run[option][k]['runs_per_day'].append(num_runs)


In [6]:
kWh_per_day = {} #{'p100-v100': {'cv': {'p100': xx, 'v100': [x]}}}
for option in energy_per_run:
    kWh_per_day[str(option)] = {}
    for model_type, val in energy_per_run[option].items():
        kWh_per_day[str(option)][model_type] = {}
        base, upgrade = option[0], option[1]
        kWh_per_day[str(option)][model_type][base] = np.mean(np.array(val[base]) * np.array(val['runs_per_day']))
        kWh_per_day[str(option)][model_type][upgrade] = np.mean(np.array(val[upgrade]) * np.array(val['runs_per_day']))
with open('operation_carbon_kWh_per_day.json', 'w') as f:
    json.dump(kWh_per_day, f, indent=4)        

In [7]:
kWh_per_day

{"('p100', 'v100')": {'candle': {'p100': 1.9188005377049862,
   'v100': 1.55106678768845},
  'cv': {'p100': 2.297810571741488, 'v100': 1.8487037611495065},
  'nlp': {'p100': 2.345773592819839, 'v100': 1.8336973008366404}},
 "('p100', 'a100')": {'candle': {'p100': 1.9188005377049862,
   'a100': 0.7565739983530946},
  'cv': {'p100': 2.297810571741488, 'a100': 0.9873210009293063},
  'nlp': {'p100': 2.345773592819839, 'a100': 1.255950342906979}},
 "('v100', 'a100')": {'candle': {'v100': 2.918078612656404,
   'a100': 1.3715330831131818},
  'cv': {'v100': 3.2541994069663644, 'a100': 1.6866693977075802},
  'nlp': {'v100': 3.3337250272624828, 'a100': 2.2785614722888266}}}